In [ ]:
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.advanced_activations import ELU
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras.optimizers import Adam
from keras.callbacks import CSVLogger, ModelCheckpoint, EarlyStopping
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras import layers
import pandas as pd
import cv2
import numpy as np

### Загрузим данные

In [ ]:
import pandas as pd
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
 
dataset_path = '/Users/baltachev/Documents/fer2013/fer2013/fer2013.csv'
image_size=(48,48)
 
def load_fer2013():
    data = pd.read_csv(dataset_path)
    pixels = data['pixels'].tolist()
    width, height = 48, 48
    faces = []
    for pixel_sequence in pixels:
        face = [int(pixel) for pixel in pixel_sequence.split(' ')]
        face = np.asarray(face).reshape(width, height)
        face = cv2.resize(face.astype('uint8'),image_size)
        faces.append(face.astype('float32'))
    faces = np.asarray(faces)
    faces = np.expand_dims(faces, -1)
    emotions = pd.get_dummies(data['emotion']).as_matrix()
    return faces, emotions
 
def preprocess_input(x, v2=True):
    x = x.astype('float32')
    x = x / 255.0
    if v2:
        x = x - 0.5
        x = x * 2.0
    return x
 
faces, emotions = load_fer2013()
faces = preprocess_input(faces)
xtrain, xtest, ytrain, ytest = train_test_split(faces, emotions,test_size=0.2,shuffle=True)

### Установим параметры модели

In [ ]:
# parameters
batch_size = 128
num_epochs = 15
input_shape = (48, 48, 1)
verbose = 1
num_classes = 7
patience = 5
base_path = '/Users/baltachev/Documents/face_recognition/models/'
optimizer = Adam(lr = 1e-3)

### Data augmentation

In [ ]:
# data generator
data_generator = ImageDataGenerator(
                        featurewise_center=False,
                        featurewise_std_normalization=False,
                        rotation_range=10,
                        width_shift_range=0.1,
                        height_shift_range=0.1,
                        zoom_range=.1,
                        horizontal_flip=True)

### Составим архитектуру сети
#### Используем основу VGG - сети

In [ ]:
# model parameters
model = Sequential()

#Block 1
model.add(Conv2D(32, (3, 3), kernel_initializer = "he_normal", padding="same", input_shape = input_shape))
model.add(ELU())
model.add(BatchNormalization())
model.add(Conv2D(32, (3, 3), kernel_initializer = "he_normal", padding="same"))
model.add(ELU())
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
 
#Block 2
model.add(Conv2D(64, (3, 3), kernel_initializer = "he_normal", padding="same"))
model.add(ELU())
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), kernel_initializer = "he_normal", padding="same"))
model.add(ELU())
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
 
# Block 3
model.add(Conv2D(128, (3, 3), kernel_initializer = "he_normal", padding="same"))
model.add(ELU())
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), kernel_initializer = "he_normal", padding="same"))
model.add(ELU())
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
 
# Block 4    
model.add(Flatten())
model.add(Dense(64, kernel_initializer="he_normal"))
model.add(ELU())
model.add(BatchNormalization())
model.add(Dropout(0.5))

# Block 5
model.add(Dense(64, kernel_initializer="he_normal"))
model.add(ELU())
model.add(BatchNormalization())
model.add(Dropout(0.5))

#Block 6 
model.add(Dense(num_classes, kernel_initializer="he_normal"))
model.add(Activation("softmax"))

### Настроим callbacks

In [ ]:
# callbacks
log_file_path = base_path + '_emotion_training_vgg.log'
csv_logger = CSVLogger(log_file_path, append=False)
early_stop = EarlyStopping('val_loss', patience=patience)
reduce_lr = ReduceLROnPlateau('val_loss', factor=0.1, patience=int(patience/4), verbose=1)
trained_models_path = base_path + '_vgg'
model_names = trained_models_path + '.{epoch:02d}-{val_acc:.2f}.hdf5'
model_checkpoint = ModelCheckpoint(model_names, 'val_loss', verbose=1,save_best_only=True)
callbacks = [model_checkpoint, csv_logger, early_stop, reduce_lr]
 


### Скомпилируем и обучим модель

In [ ]:
model.compile(optimizer=optimizer, loss='categorical_crossentropy',metrics=['accuracy'])
model.fit_generator(data_generator.flow(xtrain, ytrain, batch_size),
                        steps_per_epoch=len(xtrain) / batch_size,
                        epochs=num_epochs, verbose=1, callbacks=callbacks,
                        validation_data=(xtest,ytest))

После 15 эпох accuracy на валидационных данных составило 0.6101

### Проверим как работает сеть на тестовых изображениях

![title](happy_mar.png)

![title](sad_mar.png)

![title](surprised_mar.png)

![title](angry_mar.png)